In [ ]:
from kingdon import Algebra 
import numpy as np 
import jax.numpy as jnp 
import jax

In [ ]:
alg = Algebra(p=3, q=0, r=0)
locals().update(alg.blades)

In [ ]:
v = alg.multivector(name = 'v', grades = (1,))
b = alg.multivector(name = 'b', grades = (2,))
p = alg.multivector(name = 'p', grades = (2,))

((v * b - b * v) * e123) 

In [ ]:
(v.cp(b) * e123).acp(b) # gives 0
(v.cp(b) * e123) | (b) # also gives 0

In [ ]:
b.cp(p) # gives cross prod like term

In [ ]:
b.acp(p)

In [ ]:
(b.cp(p)).acp(b) # gives 0

In [ ]:
def deriv_swarm(b1, b2):
    # b1 time derivative from b2
    return b2 + b1.acp(b2) * b1

def deriv_general_bvec(b1, b2):
    return b2.cp(b1)

In [ ]:
bvec_formulas = [[.5,.5,0],[0,1,0]]
bvec = np.array(bvec_formulas).T
bvec_1 = alg.bivector(bvec[:, 0])
bvec_2 = alg.bivector(bvec[:, 1])

print(bvec_1)
print(bvec_2)

print(deriv_swarm(bvec_1, bvec_2))
print(deriv_general_bvec(bvec_1, bvec_2))

In [ ]:
dt = .001 
final_t = 10

print(bvec_1)
print(bvec_2)
bvec_1_update = bvec_1
bvec_2_update = bvec_2
for t in np.arange(0, final_t, dt):
    bvec_1_update = deriv_swarm(bvec_1_update, bvec_2) * dt + bvec_1_update
    #bvec_1_update = deriv_general_bvec(bvec_1_update, bvec_2_update) * dt + bvec_1_update
print(bvec_1_update)


In [ ]:
import sympy as sp

In [ ]:
x = sp.symbols('x')
sympy_function = x + x**3 - x**4
print(sympy_function)

In [ ]:
#swarmalator formula 
deriv_swarm = p + b.acp(p) * b 
print(deriv_swarm)

deriv_general = b.cp(p)
print(deriv_general)

# check orthogonality to b
orthogonal_swarm = (deriv_swarm | b)
print(f'orthogonal_swarm is {orthogonal_swarm}' )

# check orthogonality to b 
orthogonal_general = (deriv_general | b)
print(f'orthogonal_general is {orthogonal_general}' )

In [ ]:
T = 300 
N = 100 
D = 3 

traj_coeffs_rand = jax.random.uniform(jax.random.PRNGKey(0), shape=(N,T, 8), minval=-1, maxval=1)

rand_pos = jax.random.uniform(jax.random.PRNGKey(0), shape=(T, N,D), minval=-1, maxval=1)
rand_ori = jax.random.uniform(jax.random.PRNGKey(1), shape=(T, N,D), minval=-1, maxval=1)
ts = jnp.arange(T)



In [ ]:
import sympy as sp

## polynomial map inference GA

In [ ]:
# add path to /src/utils/polynomial_fits.py
import sys
sys.path.append('/Users/charlesxu/Documents/MIT/geometric algebra/dynamics_inference')
from src.utils.polynomial_fits import orth_poly_mc, fit_polynomial_mc


In [ ]:
from src.utils.polynomial_fits import orth_poly_mc
import importlib

importlib.reload(sys.modules['src.utils.polynomial_fits'])

In [ ]:
data_pos = np.arange(-1, 1, .01)
polys = orth_poly_mc(data_pos, 5)

In [ ]:
# plot all the polys over data_pos 
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
for poly in polys:
    
    plt.plot(data_pos, poly(data_pos), label=str(poly))
plt.title('Orthogonal Polynomials')
plt.xlabel('x')
plt.ylabel('Polynomial Value')
plt.show()

In [ ]:
def target_function(x):
    return x + x**3 - x**4 + 4 - 5 * np.sin(5 * x)

Y = target_function(data_pos)

In [ ]:
least_squares_fit = fit_polynomial_mc(data_pos, Y, polys)
least_squares_fit

In [ ]:
plt.plot(data_pos, least_squares_fit(data_pos), label='Least Squares Fit', color='red')
plt.scatter(data_pos, Y, label='Target Function', color='blue', s=10)
plt.title('Least Squares Fit to Target Function')
plt.xlabel('x')
plt.ylabel('Function Value')
plt.legend()
plt.show()


In [ ]:
# ok lets see if this will work with the metamaterial data
metamaterial_data = np.load('normalized_trajectory_metamaterial.npz')
x1 = metamaterial_data['x']
x2 = metamaterial_data['y']
A = metamaterial_data['angle']
# convert A to radians 
A = np.deg2rad(A)
final_time = 1 # seconds 
t = np.linspace(0, final_time, x1.shape[0])
# repeat t 
t = np.repeat(t[:, np.newaxis], x1.shape[1], axis=1)

In [ ]:
plt.plot(A)
plt.show()

In [ ]:
alg = Algebra(p=2, q=0, r=0)
locals().update(alg.blades)

In [ ]:
e12

In [ ]:
x1_t0 = x1[0, :][:, np.newaxis]  
repeat_x1_initial = np.repeat(x1_t0, x1.shape[0], axis=1)
x2_t0 = x2[0, :][:, np.newaxis]
repeat_x2_initial = np.repeat(x2_t0, x2.shape[0], axis=1)

# t, x1_t0, x2_t0 all need to be flattened
t = t.flatten()
repeat_x1_initial = repeat_x1_initial.flatten()
repeat_x2_initial = repeat_x2_initial.flatten()

data_pos = alg.multivector(e = t, 
                           e1 = repeat_x1_initial, 
                           e2 = repeat_x2_initial)

print(repeat_x1_initial.shape)



In [ ]:
data_pos.shape

In [ ]:
def constant_poly(x):
    return x * 0 + 1

def normalize(f, data_locations):
    norm = np.sqrt(inner_product(f, f, data_locations))
    print(norm.shape)
    return lambda x: f(x) / norm

def inner_product(f, g, data_locations):
    print(f(data_locations).shape)
    print(g(data_locations).shape)
    return np.mean(f(data_locations) * g(data_locations))


In [ ]:
data_pos.shape

In [ ]:
(constant_poly(data_pos) | constant_poly(data_pos))

In [ ]:
inner_product(constant_poly, constant_poly, data_pos)

In [ ]:
normalize(constant_poly, data_pos)

In [ ]:
polys = orth_poly_mc(data_pos, 5)

In [ ]:
Y = alg.multivector(e = t, e1 = x1, e2 = x2, e12 = A)


In [ ]:
(X * X).shape

# not really GA learning first

In [ ]:
x1_t0 = x1[0, :][:, np.newaxis]  
repeat_x1_initial = np.repeat(x1[0, :][:, np.newaxis], x1.shape[0], axis=1).T
x2_t0 = x2[0, :][:, np.newaxis]
repeat_x2_initial = np.repeat(x2[0, :][:, np.newaxis], x2.shape[0], axis=1).T
# t, x1_t0, x2_t0 all need to be flattened
t = t.flatten()
repeat_x1_initial = repeat_x1_initial.flatten()
repeat_x2_initial = repeat_x2_initial.flatten()

# data_pos = alg.multivector(e = t, 
#                            e1 = repeat_x1_initial, 
#                            e2 = repeat_x2_initial)


print(repeat_x1_initial.shape)

In [ ]:
plt.scatter(t, repeat_x1_initial, label='x1')

In [ ]:
plt.scatter(t, repeat_x2_initial)

In [ ]:
t = t.flatten()
repeat_x1_initial = repeat_x1_initial.flatten()
repeat_x2_initial = repeat_x2_initial.flatten()
A_initial = np.zeros_like(repeat_x1_initial)

X = np.array([t, repeat_x1_initial, repeat_x2_initial, A_initial]).T
print(X.shape)

Y = np.array([t, x1.flatten(), x2.flatten(), A.flatten()]).T
print(Y.shape)


 

In [ ]:
X[100]

In [ ]:
Y[100]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

def polynomial_fit(X, Y, max_degree, feature_names, alpha=0.0):
    """
    Perform polynomial regression with optional L1 sparsity (lasso).

    Parameters:
    - X: array-like, shape (n_samples, n_features)
    - Y: array-like, shape (n_samples, n_outputs)
    - max_degree: int, max polynomial degree
    - feature_names: list of str, names of input features
    - alpha: float, L1 regularization strength (0 for no sparsity)

    Returns:
    - equations: list of fitted equations as strings
    - mse_list: list of mean squared errors for each output
    - model: trained model object
    """
    poly = PolynomialFeatures(degree=max_degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    terms = poly.get_feature_names_out(feature_names)

    n_outputs = Y.shape[1]
    equations = []
    mse_list = []
    model_list = []
    pred_list = []

    for output_idx in range(n_outputs):
        y = Y[:, output_idx]

        if alpha > 0:
            model = Lasso(alpha=alpha, max_iter=10000)
        else:
            model = LinearRegression()

        model.fit(X_poly, y)
        y_pred = model.predict(X_poly)
        mse = mean_squared_error(y, y_pred)
        mse_list.append(mse)

        intercept = model.intercept_
        coefficients = model.coef_

        # Build sparse equation string
        equation_terms = [f"({intercept:.5f})"]
        for coef, term in zip(coefficients, terms):
            if np.abs(coef) > 1e-7:  # Include only nonzero coefficients
                equation_terms.append(f"({coef:.5f})*{term}")

        equation_str = " + ".join(equation_terms)
        equations.append(equation_str)

        print(f"Output '{feature_names[output_idx]}':")
        print(f"  MSE: {mse:.5f}")
        print(f"  Equation: {equation_str}\n")
        model_list.append(model)
        pred_list.append(y_pred)

    return equations, mse_list, model_list, pred_list

In [ ]:
# Example usage:
#X, Y = your data with shape (2007, 4)
feature_names = ['t', 'x1', 'x2', 'A']
equations, mses, models, preds = polynomial_fit(X, Y, max_degree=2, feature_names=feature_names, alpha=0)
t_pred, x1_pred, x2_pred, A_pred = preds[0], preds[1], preds[2], preds[3]
t_pred_reshape = t_pred.reshape((x1.shape[0], x1.shape[1]))
x1_pred_reshape = x1_pred.reshape((x1.shape[0], x1.shape[1]))
x2_pred_reshape = x2_pred.reshape((x1.shape[0], x1.shape[1]))
A_pred_reshape = A_pred.reshape((x1.shape[0], x1.shape[1]))

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.scatter(t, x1.flatten(), label='x1 Original')
plt.scatter(t_pred_reshape, x1_pred_reshape, label='x1 Predicted')
plt.title('x1 Original vs Predicted')
plt.xlabel('Time')
plt.ylabel('x1')
plt.legend()
plt.subplot(2, 2, 2)
plt.scatter(t, x2.flatten(), label='x2 Original')
plt.scatter(t, x2_pred, label='x2 Predicted')
plt.title('x2 Original vs Predicted')
plt.xlabel('Time')
plt.ylabel('x2')
plt.legend()
plt.subplot(2, 2, 3)
plt.scatter(t, A.flatten(), label='A Original')
plt.scatter(t, A_pred, label='A Predicted')
plt.title('A Original vs Predicted')
plt.xlabel('Time')
plt.ylabel('A')
plt.legend()
plt.tight_layout()
# set the figure size
plt.show()


In [ ]:
plt.scatter(t, x1)

## Next do it as membrane dynamics


### Begin with some visualization of the data

In [ ]:
def infer_grid_structure_3x3(x1, x2):
    # infers the grid structure for a 3x3 grid of points
    distances = np.sqrt((x1[:, None] - x1[None, :])**2 + (x2[:, None] - x2[None, :])**2)
    grid_structure = np.zeros_like(distances)
    max_distance = np.max(distances)
    grid_structure[distances < .4 * max_distance ] = 1
    return grid_structure

def reorder_points_natural_grid(x1,x2):
    # reorder points in a natural square grid structure
    # warp the metric to order it by rows first
    actual_ordering = np.argsort(np.linalg.norm(np.stack([x1,5 * x2]) , axis = 0) )# sort by sum of coordinates
    ordered_x1 = x1[actual_ordering]
    ordered_x2 = x2[actual_ordering]
    return ordered_x1, ordered_x2



In [ ]:
np.concat([x1_values, x2_values]).shape

In [ ]:
# Select a specific time index (e.g., t=100)
time_index = -1

# Extract x1, x2, and A values for the given time index
x1_values = x1[time_index]
x2_values = x2[time_index]
A_values = A[time_index]

# Reorder the points in a natural grid structure
ordered_x1, ordered_x2 = reorder_points_natural_grid(x1_values, x2_values)


In [ ]:

import matplotlib.pyplot as plt

# Create a figure
plt.figure(figsize=(8, 6))

# Scatter plot
scatter = plt.scatter(x1_values, x2_values, c=np.arange(len(ordered_x1)), cmap='viridis', s=100)

# Add labels for each point
for i, (x, y) in enumerate(zip(ordered_x1, ordered_x2)):
	plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

plt.colorbar(scatter, label='Point Index')
plt.title(f'Unordered Points at Time Index {time_index}')

In [ ]:
import time

import matplotlib.pyplot as plt

# Create a figure
plt.figure(figsize=(8, 6))

# Scatter plot
scatter = plt.scatter(ordered_x1, ordered_x2, c=np.arange(len(ordered_x1)), cmap='viridis', s=100)

# Add labels for each point
for i, (x, y) in enumerate(zip(ordered_x1, ordered_x2)):
	plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

plt.colorbar(scatter, label='Point Index')
plt.title(f'Ordered Points at Time Index {time_index}')

In [ ]:
import networkx as nx

In [ ]:
neighbor_list = infer_grid_structure_3x3(ordered_x1, ordered_x2)
# convert the adjacency matrix to a list of neighbors. have an option to include self loops
def adjacency_matrix_to_neighbors(adj_matrix, self_loops=False):
    if not self_loops:
        for i in range(adj_matrix.shape[0]):
            # set diagonal to 0 if self_loops is False
            adj_matrix[i,i] = 0
    neighbors = {}
    for i in range(adj_matrix.shape[0]):
        neighbors[i] = np.where(adj_matrix[i] > 0)[0].tolist()
    return neighbors

neighbors = adjacency_matrix_to_neighbors(neighbor_list)
material_graph = nx.Graph(neighbors)

In [ ]:
nx.draw(material_graph, with_labels=True, node_size=500, node_color='lightblue', font_size=10, font_color='black')

In [ ]:

pos = np.stack([x1, x2, A])
print(pos.shape)
pos = np.transpose(pos, (1,2,0))
print(pos.shape)

In [ ]:
import sys
sys.path.append('/Users/charlesxu/Documents/MIT/geometric algebra/dynamics_inference/')

from visualizer.temporal_graph_matplotlib import animate_temporal_graph 

anim = animate_temporal_graph(pos, material_graph)
anim.save('temporal_graph_animation.mp4', writer='ffmpeg', fps=10)

In [ ]:
import plotly.graph_objects as go

# Select a specific time index (e.g., t=100)
time_index = 100

# Extract x1, x2, and A values for the given time index
x1_values = x1[time_index]
x2_values = x2[time_index]
A_values = A[time_index]

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x1_values,
    y=x2_values,
    z=A_values,
    mode='markers',
    marker=dict(
        size=5,
        color=A_values,  # Set color to A values
        colorscale='Viridis',  # Choose a colorscale
        opacity=0.8
    )
)])

# Set labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='A'
    ),
    title=f'3D Plot of x1, x2, and A at t={time_index}'
)

# Show the plot
fig.show()

In [ ]:
x1_pred_reshape.shape

In [ ]:
t.shape

In [ ]:
plt.scatter(t, x1_pred_reshape);

In [ ]:
# make an animation of the trajectories, x,y and a quiver plot of the angle
# x has shape (T, N_locations), etc 
import os
import matplotlib.animation as animation

def animate_trajectories(x, y, angle, interval=100):
    fig, ax = plt.subplots()
    
    arrow_length = .2  # fixed arrow length

    # set the axes to be equal aspect ratio
    ax.set_aspect('equal', adjustable='box')
    margin = .2
    ax.set_xlim(np.min(x)-margin, np.max(x)+margin)
    ax.set_ylim(np.min(y)-margin, np.max(y)+margin)

    # turn off the grid and ticks
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Predicted')

    scatter = ax.scatter(x[0], y[0], color='blue', s=200)
    quiver = ax.quiver(x[0], y[0], 
                       np.cos(np.radians(angle[0])), 
                       np.sin(np.radians(angle[0])), 
                       angles='xy', scale_units='xy', scale=1/(arrow_length), color='r')

    def update(frame):
        scatter.set_offsets(np.column_stack((x[frame], y[frame])))
        u = np.cos(np.radians(angle[frame]))
        v = np.sin(np.radians(angle[frame]))
        quiver.set_offsets(np.column_stack((x[frame], y[frame])))
        quiver.set_UVC(u, v)
        return scatter, quiver

    ani = animation.FuncAnimation(fig, update, frames=len(x), interval=interval, blit=True)
    return ani

A_pred_reshape_degrees = np.rad2deg(A_pred_reshape)
ani = animate_trajectories(x1_pred_reshape, x2_pred_reshape, A_pred_reshape_degrees, interval=100)
# Save the animation as an MP4 file
output_dir = 'animations'
os.makedirs(output_dir, exist_ok=True)
ani.save(os.path.join(output_dir, 'trajectories_predicted.mp4'), writer='ffmpeg', fps=30)

In [ ]:
x1_pred_reshape.shape

In [ ]:
plt.plot(x1_pred_reshape);